In [5]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

In [11]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 47.4239,
	"longitude": 9.3748,
	"start_date": "2019-09-01",
	"end_date": "2023-11-09",
	"daily": ["weather_code", "temperature_2m_max", "temperature_2m_min", "rain_sum", "snowfall_sum"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()
daily_rain_sum = daily.Variables(3).ValuesAsNumpy()
daily_snowfall_sum = daily.Variables(4).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s"),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s"),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}
daily_data["weather_code"] = daily_weather_code
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["rain_sum"] = daily_rain_sum
daily_data["snowfall_sum"] = daily_snowfall_sum

daily_dataframe = pd.DataFrame(data = daily_data)
print(daily_dataframe)

Coordinates 47.41651916503906°E 9.442623138427734°N
Elevation 679.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
           date  weather_code  temperature_2m_max  temperature_2m_min  \
0    2019-09-01          63.0           21.101000           15.801000   
1    2019-09-02          63.0           15.000999           13.151000   
2    2019-09-03           2.0           18.150999            9.601000   
3    2019-09-04           1.0           21.101000           11.000999   
4    2019-09-05          63.0           15.301000           10.601000   
...         ...           ...                 ...                 ...   
1526 2023-11-05          61.0           11.301000            4.951000   
1527 2023-11-06          63.0           11.201000            2.801000   
1528 2023-11-07          55.0            9.551000            2.351000   
1529 2023-11-08           1.0           10.201000            0.301000   
1530 2023-11-09          51.0            9.951000            0.351000  

In [12]:
daily_dataframe.to_csv("C:/Users/rita_/OneDrive/Dokumente/git/parcaster/data/scraping/weather/historical_data/historical_weather.csv"
                       , sep=","
                       , index=False)